# Ordinary Least Squares

In [31]:
import pandas
import statsmodels.api as sm_api
import statsmodels.formula.api as sm
import scipy.stats as stats
import numpy as np

In [26]:
data = pandas.read_csv('washingtonbikeshare.csv')
data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [27]:
data_train = data.loc[(data['yr'] == 0) & (data['mnth'] < 10)]
data_test = data.loc[(data['yr'] == 0) & (data['mnth'] >= 10)]

In [28]:
model = sm.ols(formula="cnt ~ atemp + temp + hum + windspeed", data=data_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.744
Method:                 Least Squares   F-statistic:                     198.4
Date:                Mon, 04 Mar 2024   Prob (F-statistic):           8.06e-79
Time:                        16:13:35   Log-Likelihood:                -2190.2
No. Observations:                 273   AIC:                             4390.
Df Residuals:                     268   BIC:                             4408.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1708.8451    296.956      5.755      0.000    1124.182    2293.509
atemp      -3132.5570   3164.040     -0.990      0.323   -9362.093    3096.979
temp        8823.6644   2823.617      3.125      0.002    3264.372    1.44e+04
hum        -1134.9410    302.778     -3.748      0.000   -1731.067    -538.815
windspeed  -3052.9184    642.368     -4.753      0.000   -4317.647   -1788.190
==============================================================================
Omnibus:                       15.311   Durbin-Watson:                   0.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.768
Skew:                          -0.383   Prob(JB):                     1.14e-05
Kurtosis:                       4.190   Cond. No.                         132.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
# Number of observations
n = len(data_train)

# Number of predictors
p = 4

# Total sum of squares (y - y_bar)^2
sst = ((data_train['cnt'] - data_train['cnt'].mean()) ** 2).sum()
df_total = n - 1

# Error sum of squares (y - y_hat)^2
sse = ((data_train['cnt'] - model.predict()) ** 2).sum()
df_error = n - p - 1

# Regression sum of squares (y_hat - y_bar)^2
ssr = ((model.predict() - data_train['cnt'].mean()) ** 2).sum()

assert(np.isclose(sst, sse + ssr))

# R-squared:
rsquared = 1 - sse / sst
assert(np.isclose(model.rsquared, rsquared))

# Adjusted R-squared:
adj_rsquared = 1 - (sse / df_error) / (sst / df_total)
assert(np.isclose(model.rsquared_adj, adj_rsquared))

# F-test:
f_test = (ssr / p) / (sse / df_error)
assert(np.isclose(model.fvalue, f_test))

# Pr(F_{p, n - p} > F_test):
f_pvalue = 1 - stats.f.cdf(f_test, p, n - p)
assert(np.isclose(model.f_pvalue, f_pvalue)) # Note that this value is extremely small, and may suffer from numerical instability

# Log-likelihood:
log_likelihood = -n / 2 * (np.log(2 * np.pi) + np.log(sse / n) + 1)
assert(np.isclose(model.llf, log_likelihood))

# Akaike Information Criteria:
aic = 2 * (p + 1) - 2 * log_likelihood
assert(np.isclose(model.aic, aic))

# Bayesian Information Criteria:
bic = (p + 1) * np.log(n) - 2 * log_likelihood
assert(np.isclose(model.bic, bic))

In [59]:
# Covariance matrix of the residuals
X = sm_api.add_constant(data_train[['atemp', 'temp', 'hum', 'windspeed']])
cov_matrix = np.linalg.inv(np.dot(X.T, X))

# Standard errors for each predictor
se_predictors = np.sqrt(np.diagonal(sse / df_error * cov_matrix))
assert(np.isclose(model.bse, se_predictors).all())

# T-statistics for each predictor
t_stats = model.params / se_predictors
assert(np.isclose(model.tvalues, t_stats).all())

# Pr(|T_{n - p - 1}| > |t_{n - p - 1}|):
t_pvalues = 2 * (1 - stats.t.cdf(np.abs(t_stats), df_error))
assert(np.isclose(model.pvalues, t_pvalues).all())

# Confidence intervals for each predictor
conf_int = np.vstack((
    model.params - stats.t.ppf(0.975, df_error) * se_predictors,
    model.params + stats.t.ppf(0.975, df_error) * se_predictors
)).T

assert(np.isclose(model.conf_int(), conf_int).all())

In [61]:
# Root Mean Squared Error
rmse = np.sqrt(sse / n)
print(rmse)

737.8884543736128
